# ExaGO mpi4py Jupyter Tutorial

We have a Python wrapper for ExaGO installed in this container through sapck. You can view the Dockerfile in `.devcontainer/Dockerfile`. See the Appendix for more details.

This is different to the base tutorial in that we include some additional information about MPI configuration.

In [1]:
!mpirun -n 5 opflow --version

ExaGO 1.6.0 built on Nov  9 2023
ExaGO 1.6.0 built on Nov  9 2023
ExaGO 1.6.0 built on Nov  9 2023
ExaGO 1.6.0 built on Nov  9 2023
ExaGO 1.6.0 built on Nov  9 2023


In [2]:
# Unique to MPI configuration
print(f'Hello')
import mpi4py
mpi4py.rc.threads = False
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(f'{comm} from rank {rank} out of {size} total ranks')

Hello
<mpi4py.MPI.Intracomm object at 0x7fd664469910> from rank 0 out of 1 total ranks


In [3]:
# Get pwd and change directory to where datafiles are
import os
print(f'{os.getcwd()}')
os.chdir('../..')
print(f'{os.getcwd()}')

/home/app/docs/devcontainer
/home/app


We need to run in a parallel cluster, save file to disk (or return in place), and then do something with the solution...

In [4]:
# Hello world first:
import ipyparallel as ipp

def mpi_example():
    import os
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    return f"Hello World from rank {comm.Get_rank()}. total ranks={comm.Get_size()}. host={MPI.Get_processor_name()} out of {os.getcwd()}"

# request an MPI cluster with 24 engines
with ipp.Cluster(controller_ip="*", engines="mpi", n=6) as rc:
    # get a broadcast_view on the cluster which is best
    # suited for MPI style computation
    view = rc.broadcast_view()
    # run the mpi_example function on all engines in parallel
    r = view.apply_sync(mpi_example)
    # Retrieve and print the result from the engines
    print("\n".join(r))
# at this point, the cluster processes have been shutdown

Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|██████████| 6/6 [00:05<00:00,  1.03engine/s]
Hello World from rank 0. total ranks=6. host=dfdb9932ea09 out of /home/app
Hello World from rank 1. total ranks=6. host=dfdb9932ea09 out of /home/app
Hello World from rank 2. total ranks=6. host=dfdb9932ea09 out of /home/app
Hello World from rank 3. total ranks=6. host=dfdb9932ea09 out of /home/app
Hello World from rank 4. total ranks=6. host=dfdb9932ea09 out of /home/app
Hello World from rank 5. total ranks=6. host=dfdb9932ea09 out of /home/app
Stopping engine(s): 1699501339
engine set stopped 1699501339: {'exit_code': 0, 'pid': 35617, 'identifier': 'ipengine-1699501338-ciod-1699501339-35554'}
Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 35587, 'identifier': 'ipcontroller-1699501338-ciod-35554'}


In [5]:
import ipyparallel as ipp
import os
print(f'{os.getcwd()}')
os.chdir('/home/app')
print(f'{os.getcwd()}')

def run_exago():
    from mpi4py import MPI
    import exago
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    exago.initialize("app", comm)
    opf = exago.OPFLOW()
    opf.read_mat_power_data('./datafiles/case9/case9mod.m')
    opf.solve()

    comm.barrier()

    if rank == 0:
        # opf.print_solution()
        opf.save_solution(exago.OutputFormat.JSON, 'solution')

    # exago.finalize()
    comm.barrier()

# request an MPI cluster with 24 engines
with ipp.Cluster(controller_ip="*", engines="mpi", n=4) as rc:
    # get a broadcast_view on the cluster which is best
    # suited for MPI style computation
    view = rc.broadcast_view()
    # run the mpi_example function on all engines in parallel
    r = view.apply_sync(run_exago)

/home/app
/home/app
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|██████████| 4/4 [00:05<00:00,  1.48s/engine]
Stopping engine(s): 1699501347
engine set stopped 1699501347: {'exit_code': 59, 'pid': 35775, 'identifier': 'ipengine-1699501346-0x7o-1699501347-35554'}
Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 35742, 'identifier': 'ipcontroller-1699501346-0x7o-35554'}


In [16]:
!cat solution.json

{
	"casefile": "./datafiles/case9/case9mod.m",
	"gicfile": "not given",
	"nbranch": 9,
	"ngen": 3,
	"nbus": 9,
	"KVlevels": [
		345.000000
	],
	"casejsonfile": "solution.json",
	"geojsondata": {
		"type": "FeatureCollection",
		"features": [
			{
				"type": "Feature",
				"geometry": {
					"type": "Point",
					"coordinates": [
						-97.508469,
						35.981918
					]
				},
				"properties": {
					"elementtype": "SUBSTATION",
					"NAME": "1",
					"nbus": 1,
					"KVlevels": [
						345.000000
					],
					"bus": [
						{
							"elementtype": "bus",
							"BUS_I": 1,
							"VA": 0.000000,
							"VM": 1.099983,
							"BUS_NAME": " 1",
							"VMIN": 0.900000,
							"VMAX": 1.100000,
							"BASE_KV": 345.000000,
							"PD": 0.000000,
							"QD": 0.000000,
							"PDloss": 0.000000,
							"QDloss": 0.000000,
							"LAM_P": 2102.907811,
							"LAM_Q": 0.000000,
							"ngen": 1,
							"gen": [
								{
									"GEN_BUS": 1,
									"GEN_FUEL": "UNDEFINED",
									"P

The output to stdout isn't quite right, and we noteably don't call `exago.finalize()` which is a little concerning, but this seems to work. Should be fine to run SCOPFLOW and more complex applications, and then you can loop over ExaGO runs like this all natively to Python!

## MPI Execution Options

There are quite a few ways to run MPI backed code, and here are some notes on how I arrived at the above implementation.

In [6]:
!cat /usr/local/share/jupyter/kernels/py311-mpi4py-exago/kernel.json

{
 "argv": [
  "mpiexec", "-n", "1", "/opt/views/view/bin/python3",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "ExaGO w/ MPI",
 "language": "python",
 "metadata": {
  "debugger": true
 }
}

Firstly we execute the ipykernel backend using MPI. This means all kernels are launched within a single rank MPI context, and MPI basics work.

To run with more processes, we can either:
1. Use `!` in a Python block and just run with `mpiexec -n N` for the number of ranks
    - This works, but I/O is clunky
    - It becomes cumbersome to code dynamic high-level applications that re-run different sims based on returned output
1. Spawn an ipyparallel cluster
    - https://ipyparallel.readthedocs.io/en/latest/reference/mpi.html 
        - you can start the cluster and connect to it, but these docs are a little confusing, as you start many processes
        - Would be better to have it all be jupyter native
    - https://kb.oakland.edu/uts/HPCJupyterMPI
        - Inlcudes a much better Jupyter focused bit of docs around this
1. Use a different workflow engine, and find a tool

While a workflow tool is a strong consideration for most high level applications, we are going to stick with 2. 1 and 3 would likely be custom to your use-case

# Appendix

From https://github.com/mpi4py/mpi4py/issues/15#issuecomment-765658147, had to modify mpi4py init routines by hand - this was all due to a red-herring, but is interesting discussion.

In [7]:
!jupyter kernelspec list

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Available kernels:
  python3               /opt/views/view/share/jupyter/kernels/python3
  py311-exago           /usr/local/share/jupyter/kernels/py311-exago
  py311-mpi4py-exago    /usr/local/share/jupyter/kernels/py311-mpi4py-exago


In [8]:
!cat /usr/local/share/jupyter/kernels/py311-exago/kernel.json

{
 "argv": [
  "/opt/views/view/bin/python3",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "ExaGO",
 "language": "python",
 "metadata": {
  "debugger": true
 }
}

As you can see, we have some extra script in our dockerfile that slightly modifies the installed Jupyter kernel to use mpiexec when launching!

In [9]:
!pip3 list

Package                 Version
----------------------- ---------
aiohttp                 3.8.4
aiosignal               1.3.1
anyio                   3.6.2
argon2-cffi             21.3.0
argon2-cffi-bindings    21.2.0
asttokens               2.4.0
async-timeout           4.0.2
attrs                   23.1.0
Babel                   2.12.1
backcall                0.2.0
beautifulsoup4          4.12.2
bleach                  6.0.0
blinker                 1.6.2
certifi                 2023.7.22
cffi                    1.15.1
charset-normalizer      3.3.0
click                   8.1.5
colorama                0.4.6
comm                    0.1.4
config                  0.5.1
dataclasses-json        0.5.9
debugpy                 1.6.7
decorator               5.1.1
defusedxml              0.7.1
entrypoints             0.4
executing               1.2.0
fastjsonschema          2.16.3
Flask                   2.3.2
Flask-Cors              4.0.0
frozenlist              1.4.0
gevent                  2

That's odd - ExaGO can import and run, but doesn't show up in pip. That's because we installed using Spack, and ExaGO is available directly through the `PYTHONPATH` environment variable.

This is configured through `entrypoint.sh`, which is the last thing that happens in the configured `~/.bashrc`:

In [10]:
!cat ~/.bashrc | tail -n 10

            fi; \
        fi`'
    local lightblue='\[\033\]'
    local removecolor='\[\033\]'
    PS1="${userpart} ${lightblue}\w ${gitbranch}${removecolor}\$ "
    unset -f __bash_prompt
}
__bash_prompt
export PROMPT_DIRTRIM=4
source /entrypoint.sh


In [11]:
!cat /entrypoint.sh

#!/bin/sh
. /opt/spack-environment/activate.sh
exec "$@"


In [12]:
!cat /opt/spack-environment/activate.sh

export SPACK_ENV=/opt/spack-environment;
export SPACK_ENV_VIEW=default;
alias despacktivate='spack env deactivate';
export ACLOCAL_PATH=/opt/views/view/share/aclocal;
export CMAKE_PREFIX_PATH=/opt/views/view;
export C_INCLUDE_PATH=/opt/views/view/include;
export JUPYTERLAB_DIR=/opt/views/view/share/jupyter/lab;
export JUPYTER_PATH=/opt/views/view/share/jupyter;
export LD_LIBRARY_PATH=/opt/views/view/lib64:/opt/views/view/lib;
export LIBRARY_PATH=/opt/views/view/lib;
export MANPATH=/opt/views/view/share/man:/opt/views/view/man:;
export MPICC=/opt/views/view/bin/mpicc;
export MPICXX=/opt/views/view/bin/mpic++;
export MPIF77=/opt/views/view/bin/mpif77;
export MPIF90=/opt/views/view/bin/mpif90;
export PATH=/opt/views/view/bin:/opt/spack/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin;
export PETSC_ARCH="";
export PETSC_DIR=/opt/views/view;
export PKG_CONFIG_PATH=/opt/views/view/lib64/pkgconfig:/opt/views/view/lib/pkgconfig:/opt/views/view/share/pkgconfig;
export PYTHONPATH

In [13]:
!echo $PYTHONPATH

/opt/views/view/lib/python3.11/site-packages


In [14]:
!ls -al /opt/views/view/lib/python3.11/site-packages

total 748
drwxr-xr-x 168 root root 12288 Nov  9 01:24 .
drwxr-xr-x  39 root root  4096 Nov  9 01:24 ..
drwxr-xr-x   2 root root  4096 Nov  9 01:24 Babel-2.12.1.dist-info
drwxr-xr-x  11 root root  4096 Nov  9 01:24 IPython
drwxr-xr-x   2 root root  4096 Nov  9 01:24 Jinja2-3.1.2.dist-info
drwxr-xr-x   2 root root  4096 Nov  9 01:24 MarkupSafe-2.1.3.dist-info
drwxr-xr-x   2 root root  4096 Nov  9 01:24 Pygments-2.16.1.dist-info
lrwxrwxrwx   1 root root   139 Nov  9 01:24 README.txt -> /opt/software/linux-ubuntu22.04-x86_64_v3/gcc-11.4.0/python-3.11.6-ybsi7tlnssr3m4h3eqrzq6tyxdcsr2n7/lib/python3.11/site-packages/README.txt
drwxr-xr-x   2 root root  4096 Nov  9 01:24 Send2Trash-1.8.0.dist-info
drwxr-xr-x   2 root root  4096 Nov  9 01:24 __pycache__
drwxr-xr-x   3 root root  4096 Nov  9 01:24 _argon2_cffi_bindings
lrwxrwxrwx   1 root root   175 Nov  9 01:24 _cffi_backend.cpython-311-x86_64-linux-gnu.so -> /opt/software/linux-ubuntu22.04-x86_64_v3/gcc-11.4.0/py-cffi-1.15.1-ns7kiujztzymljhmkw

Here you can see the full list of spack configured python packages, including exago.

## Some info about this container

In [15]:
!cat /etc/os-release

PRETTY_NAME="Debian GNU/Linux 12 (bookworm)"
NAME="Debian GNU/Linux"
VERSION_ID="12"
VERSION="12 (bookworm)"
VERSION_CODENAME=bookworm
ID=debian
HOME_URL="https://www.debian.org/"
SUPPORT_URL="https://www.debian.org/support"
BUG_REPORT_URL="https://bugs.debian.org/"
